### Load Data

In [1]:
model_name = 'model_siamese_071222'
num_epochs = 20

In [2]:
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.loader import DenseDataLoader #To make use of this data loader, all graph attributes in the dataset need to have the same shape. In particular, this data loader should only be used when working with dense adjacency matrices.
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
from f_visualization_functions import visualize_points
from torch_geometric.utils import dense_to_sparse
import math

In [3]:
data_dir_1 = 'C:/Users/david/pyproj/pyg/adl/patch_label_1'
data_dir_0 = 'C:/Users/david/pyproj/pyg/adl/patch_label_0'

In [4]:
from c_PatchDataset import PatchDataset
dataset = PatchDataset(data_dir_label_0 = data_dir_0,  data_dir_label_1=data_dir_1,  neg_pos_ratio=1)
len(dataset)

572

In [5]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs pairs: {len(dataset)}')

data = dataset[0]  # Get the first graph object.
print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes in each: {data.num_nodes}')
print(f'Number of node features: {data.num_node_features}')


Dataset: PatchDataset(572):
Number of graphs pairs: 572

PairData(adj1=[100, 100], x1=[100, 3], adj2=[100, 100], x2=[100, 3], y=1)
Number of nodes in each: None
Number of node features: 0


c:\Users\david\pyproj\pyg\pyg_env\lib\site-packages\torch_geometric\data\storage.py:271: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'x1', 'adj2', 'x2', 'adj1', 'y'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [6]:
data.adj2

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [7]:
# Does not work we do not have pos
#visualize_points(data.pos, data.edge_index)

In [8]:
from torch_geometric.loader import DataLoader 

batch_size = 1

n_train = math.ceil((4/6) * len(dataset))
n_val = math.ceil((len(dataset) - n_train)/2)
n_test = len(dataset) - n_train - n_val

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])
print(f'Number of training graphs pairs: {len(train_dataset)}')
print(f'Number of validation graphs: {len(val_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

train_loader = DataLoader(dataset = train_dataset, batch_size= batch_size, shuffle=True)
val_loader = DataLoader(dataset = val_dataset, batch_size= batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size= batch_size, shuffle=True)

Number of training graphs pairs: 382
Number of validation graphs: 95
Number of test graphs: 95


In [9]:
databatch = next(iter(train_loader))
databatch

PairDataBatch(adj1=[100, 100], x1=[100, 3], adj2=[100, 100], x2=[100, 3], y=[1])

### Define Network

In [10]:
class GNN(torch.nn.Module):
    def __init__(self, in_nodes, in_channels, hidden_channels, out_channels,
                 normalize=False, lin=True):
        super(GNN, self).__init__()

        # Each instance of this GNN will have 3 convolutional layers and three batch norm layers        
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bns1 = torch.nn.BatchNorm1d(in_nodes)
        
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bns2 = torch.nn.BatchNorm1d(in_nodes)
        
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bns3 = torch.nn.BatchNorm1d(in_nodes)


    def forward(self, x, adj, mask=None):
        
        #Step 1
        x = self.conv1(x, adj, mask)
        x = self.bns1(x)
        
        #Step 2
        x = self.conv2(x, adj, mask)
        x = self.bns2(x)

        #Step 3
        x = self.conv3(x, adj, mask)
        if x.shape[2] != 1: 
            x = self.bns3(x)

        return x


class DiffPool(torch.nn.Module):
    def __init__(self, num_nodes):
        super(DiffPool, self).__init__()

        #Hierarchical Step #1
        in_nodes = num_nodes
        out_nodes = 25 # Number of clusters / nodes in the next layer
        self.gnn1_pool = GNN(in_nodes, dataset.num_features, 16, out_nodes) # PoolGNN --> Cluster Assignment Matrix to reduce to num_nodes
        self.gnn1_embed = GNN(in_nodes, dataset.num_features, 8, 8) # EmbGNN --> Convolutions to create new node embedding

        # Hierarchical Step #2
        in_nodes = out_nodes
        out_nodes = 10
        self.gnn2_pool = GNN(in_nodes, 8, 8, out_nodes)
        self.gnn2_embed = GNN(in_nodes, 8, 12, 16, lin=False)

        # Hierarchical Step #3
        in_nodes = out_nodes
        out_nodes = 1
        self.gnn3_pool = GNN(in_nodes, 16, 16, out_nodes)
        self.gnn3_embed = GNN(in_nodes, 16, 16, 32, lin=False)

        # Final Classifier
        self.lin1 = torch.nn.Linear(32, 64) 
        #self.lin2 = torch.nn.Linear(64, 2)



    def forward(self, x, adj, batch, mask=None):
        
        #if batch == 0: print('Shape of input data batch:')
        #if batch == 0: print(f'Feature Matrix: {tuple(x.shape)}')
        #if batch == 0: print(f'Adjacency Matrix: {tuple(adj.shape)}')
       


        #Hierarchical Step #1
        #if batch == 0: print('Hierarchical Step #1')
        x1 = self.gnn1_embed(x, adj, mask) # node feature embedding
        s = self.gnn1_pool(x, adj, mask) # cluster assignment matrix

        #if batch == 0: print(f'X1 = {tuple(x1.shape)}    S1: {tuple(s.shape)}')

        x, adj, l1, e1 = dense_diff_pool(x1, adj, s, mask) # does the necessary matrix multiplications
        adj = torch.softmax(adj, dim=-1)

        #if batch == 0: print(f'---matmul---> New feature matrix (softmax(s_0.t()) @ z_0) = {tuple(x.shape)}')
        #if batch == 0: print(f'---matmul---> New adjacency matrix (s_0.t() @ adj_0 @ s_0) = {tuple(adj.shape)}')
   


        # Hierarchical Step #2
        #if batch == 0: print('Hierarchical Step #2')
        x2 = self.gnn2_embed(x, adj)
        s = self.gnn2_pool(x, adj)

        #if batch == 0: print(f'X2: {tuple(x2.shape)}    S2: {tuple(s.shape)}')
        
        x, adj, l2, e2 = dense_diff_pool(x2, adj, s)
        adj = torch.softmax(adj, dim=-1)

        #if batch == 0: print(f'---matmul---> New feature matrix (softmax(s_0.t()) @ z_0) = {tuple(x.shape)}')
        #if batch == 0: print(f'---matmul---> New adjacency matrix (s_0.t() @ adj_0 @ s_0) = {tuple(adj.shape)}')
      
        

        # Hierarchical Step #3
        #if batch == 0: print('Hierarchical Step #3')
        x3 = self.gnn3_embed(x, adj)
        s = self.gnn3_pool(x, adj)
        
        #if batch == 0: print(f'X3: {tuple(x3.shape)}    S3: {tuple(s.shape)}')

        x, adj, l3, e3 = dense_diff_pool(x3, adj, s)
        adj = torch.softmax(adj, dim=-1)

        #if batch == 0: print(f'---matmul---> New feature matrix (softmax(s_0.t()) @ z_0) = {tuple(x.shape)}')
        #if batch == 0: print(f'---matmul---> New adjacency matrix (s_0.t() @ adj_0 @ s_0) = {tuple(adj.shape)}')
     
        

        # Final Classification
        #if batch == 0: print('Final Output')
        x = x.mean(dim=1) # Pool the features of all nodes (global mean pool)  dim = 1 refers to columns
        #if batch == 0: print(f'---X Output after mean= {tuple(x.shape)}')

        x = F.relu(self.lin1(x)) # Fully connected layer + relu
        #if batch == 0: print(f'------ X Output 3 after lin= {tuple(x.shape)}')

        
        return x, l1 + l2 + l3, e1 + e2 + e3

In [11]:
class ContrastiveLoss(torch.nn.Module):

    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, x0, x1, y):
        # euclidian distance
        diff = x0 - x1
        dist_sq = torch.sum(torch.pow(diff, 2), 1)
        dist = torch.sqrt(dist_sq)

        mdist = self.margin - dist
        dist = torch.clamp(mdist, min=0.0)
        loss = y * dist_sq + (1 - y) * torch.pow(dist, 2)
        loss = torch.sum(loss) / 2.0 / x0.size()[0]
        return loss

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = DiffPool(num_nodes = 100).to(device)
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train(epoch):
    batch = 0
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output1, _, _ = model(data.x1, data.adj2, batch)
        output2, _, _ = model(data.x2, data.adj2, batch = None)
        
        #Contrastive Loss
        loss_contrastive = criterion(output1,output2,data.y)
        loss_contrastive.backward()
        loss_all += data.y.size(0) * loss_contrastive.item()
        optimizer.step()
        batch +=1

    return loss_all / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    dist_vs_label = [[],[],[]]
    

    for data in loader:
        data = data.to(device)
        output1, _, _ = model(data.x1, data.adj2, batch=None)
        output2, _, _ = model(data.x2, data.adj2, batch=None)

        test_loss_contrastive = criterion(output1, output2, data.y)
        
        euclidean_distance = F.pairwise_distance(output1, output2)
        label = data.y

        dist_vs_label[0].append(float(euclidean_distance))
        dist_vs_label[1].append(int(label))
        dist_vs_label[2].append(float(test_loss_contrastive))

    return dist_vs_label



train_results = []
validation_results = []
test_results = []

for epoch in range(1,num_epochs+1):
    
    train_loss = train(epoch)

    train_results.append(test(train_loader))
    validation_results.append(test(val_loader))
    test_results.append(test(test_loader))


    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.3f}')
    #Train Acc: {train_acc:.3f}, f'Val Acc: {val_acc:.3f}, Test Acc: {test_acc:.3f}')
    


cpu
Epoch: 001, Train Loss: 20.672
Epoch: 002, Train Loss: 0.959
Epoch: 003, Train Loss: 0.139
Epoch: 004, Train Loss: 0.145
Epoch: 005, Train Loss: 0.182
Epoch: 006, Train Loss: 0.197
Epoch: 007, Train Loss: 0.204
Epoch: 008, Train Loss: 0.203
Epoch: 009, Train Loss: 0.166
Epoch: 010, Train Loss: 0.153
Epoch: 011, Train Loss: 0.149
Epoch: 012, Train Loss: 0.145
Epoch: 013, Train Loss: 0.137
Epoch: 014, Train Loss: 0.158
Epoch: 015, Train Loss: 0.128
Epoch: 016, Train Loss: 0.135
Epoch: 017, Train Loss: 0.128
Epoch: 018, Train Loss: 0.125
Epoch: 019, Train Loss: 0.122
Epoch: 020, Train Loss: 0.116


In [13]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
gnn1_pool.conv1.bias 	 torch.Size([16])
gnn1_pool.conv1.lin.weight 	 torch.Size([16, 3])
gnn1_pool.bns1.weight 	 torch.Size([100])
gnn1_pool.bns1.bias 	 torch.Size([100])
gnn1_pool.bns1.running_mean 	 torch.Size([100])
gnn1_pool.bns1.running_var 	 torch.Size([100])
gnn1_pool.bns1.num_batches_tracked 	 torch.Size([])
gnn1_pool.conv2.bias 	 torch.Size([16])
gnn1_pool.conv2.lin.weight 	 torch.Size([16, 16])
gnn1_pool.bns2.weight 	 torch.Size([100])
gnn1_pool.bns2.bias 	 torch.Size([100])
gnn1_pool.bns2.running_mean 	 torch.Size([100])
gnn1_pool.bns2.running_var 	 torch.Size([100])
gnn1_pool.bns2.num_batches_tracked 	 torch.Size([])
gnn1_pool.conv3.bias 	 torch.Size([25])
gnn1_pool.conv3.lin.weight 	 torch.Size([25, 16])
gnn1_pool.bns3.weight 	 torch.Size([100])
gnn1_pool.bns3.bias 	 torch.Size([100])
gnn1_pool.bns3.running_mean 	 torch.Size([100])
gnn1_pool.bns3.running_var 	 torch.Size([100])
gnn1_pool.bns3.num_batches_tracked 	 torch.Size([])
gnn1_embed.conv1.bias 	 

In [14]:
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Optimizer's state_dict:
state 	 {0: {'step': tensor(7640.), 'exp_avg': tensor([ 0.0065, -0.0005, -0.0021,  0.0079,  0.0024,  0.0042, -0.0068, -0.0012,
        -0.0019,  0.0031, -0.0036, -0.0060, -0.0047,  0.0014,  0.0006,  0.0008]), 'exp_avg_sq': tensor([0.0555, 0.1313, 0.8340, 0.3583, 1.3282, 0.7502, 0.8349, 0.4627, 0.1023,
        0.5334, 0.1354, 0.7796, 0.7847, 0.8163, 0.5900, 0.2142])}, 1: {'step': tensor(7640.), 'exp_avg': tensor([[-3.9503e-03, -3.1667e-04, -2.1321e-03],
        [ 2.1602e-04, -1.1916e-05,  4.7036e-04],
        [ 3.1531e-04, -1.7319e-04,  1.9762e-03],
        [-6.2916e-03, -4.8046e-04, -1.7176e-03],
        [-2.0036e-03,  2.0898e-06, -1.2727e-03],
        [-2.4162e-03, -2.6839e-04, -1.2445e-03],
        [ 3.8874e-03, -2.6024e-05,  2.5636e-03],
        [ 8.9455e-04,  3.4127e-04, -1.6206e-03],
        [ 2.2685e-03,  2.9986e-04, -7.2458e-04],
        [-2.1362e-03, -1.4805e-04,  1.3517e-04],
        [ 2.0748e-03,  1.6566e-04,  4.2618e-04],
        [ 4.7392e-03,  2.2129

In [18]:
torch.save(model.state_dict(), f'{model_name}_state_dict.pt')

In [19]:
torch.save(model, model_name+'.pt')

In [ ]:
train_results = []
validation_results = []
test_results = []

In [22]:
train_results

[[[tensor([37.5368]),
   tensor([144.1978]),
   tensor([161.7486]),
   tensor([55.5483]),
   tensor([343.4700]),
   tensor([75.6800]),
   tensor([50.3611]),
   tensor([55.7876]),
   tensor([355.8751]),
   tensor([50.4336]),
   tensor([47.8749]),
   tensor([86.8261]),
   tensor([96.5690]),
   tensor([25.5092]),
   tensor([11.0556]),
   tensor([3.0033]),
   tensor([205.1449]),
   tensor([2.7681]),
   tensor([97.5519]),
   tensor([76.4211]),
   tensor([45.4229]),
   tensor([15.8458]),
   tensor([27.9976]),
   tensor([7.9033]),
   tensor([59.3916]),
   tensor([22.9871]),
   tensor([34.5406]),
   tensor([14.1640]),
   tensor([6.7655]),
   tensor([5.1749]),
   tensor([62.9849]),
   tensor([86.1800]),
   tensor([24.3889]),
   tensor([62.9835]),
   tensor([6.1565]),
   tensor([35.2460]),
   tensor([56.1371]),
   tensor([65.9417]),
   tensor([3.0409]),
   tensor([167.5297]),
   tensor([68.7376]),
   tensor([24.9418]),
   tensor([7.8645]),
   tensor([24.8644]),
   tensor([2.7448]),
   tensor([18

In [17]:
import numpy as np
np.save(f'{model_name}_training_loss.npy', tr_loss, allow_pickle=True)
np.save(f'{model_name}_training_accuracy.npy', tr_acc, allow_pickle=True)

np.save(f'{model_name}_validation_loss.npy', v_loss, allow_pickle=True)
np.save(f'{model_name}_validation_accuracy.npy', v_acc, allow_pickle=True)

np.save(f'{model_name}_test_loss.npy', tst_loss, allow_pickle=True)
np.save(f'{model_name}_test_accuracy.npy', tst_acc, allow_pickle=True)

NameError: name 'tr_loss' is not defined

In [ ]:
import numpy as np
training_loss = np.load(f'{model_name}_training_loss.npy', allow_pickle=True)
training_accuracy = np.load(f'{model_name}_training_accuracy.npy', allow_pickle=True)

validation_loss = np.load(f'{model_name}_validation_loss.npy', allow_pickle=True)
validation_accuracy = np.load(f'{model_name}_validation_accuracy.npy', allow_pickle=True)

test_loss = np.load(f'{model_name}_test_loss.npy', allow_pickle=True)
test_accuracy = np.load(f'{model_name}_test_accuracy.npy', allow_pickle=True)

In [ ]:
import matplotlib.pyplot as plt

# Generate a sequence of integers to represent the epoch numbers
epochs = range(1, num_epochs+1)
 
# Plot and label the training and validation loss values
plt.plot(epochs, training_loss, label='Training Loss')
plt.plot(epochs, validation_loss, label='Validation Loss')
 
# Add in a title and axes labels
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
 
# Set the tick locations
plt.xticks(np.arange(0, num_epochs+1, num_epochs/10))
 
# Display the plot
plt.legend(loc='best')
plt.savefig(f'{model_name}_Training and Validation Loss.png')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Generate a sequence of integers to represent the epoch numbers
epochs = range(1, num_epochs+1)
 
# Plot and label the training and validation loss values
plt.plot(epochs, training_accuracy, label='Training Accuracy')
plt.plot(epochs, validation_accuracy, label='Validation Accuracy')
plt.plot(epochs, test_accuracy, label='Test Accuracy')
 
# Add in a title and axes labels
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
 
# Set the tick locations
plt.xticks(np.arange(0, num_epochs+1, num_epochs/10))
plt.ylim(0,1)
 
# Display the plot
plt.legend(loc='best')
plt.savefig(f'{model_name}_Accuracy vs. Epochs.png')
plt.show()

In [ ]:
dataset_fraction = [0,0]

train_fraction = [0,0]
val_fraction = [0,0]
test_fraction = [0,0]

for grph in train_dataset: 
    if grph.y == 1: 
        train_fraction[1] +=1
        dataset_fraction[1] +=1 
    else: 
        train_fraction[0] +=1
        dataset_fraction[0] +=1 

for grph in val_dataset: 
    if grph.y == 1:
         val_fraction[1] +=1
         dataset_fraction[1] +=1  
    else:
         val_fraction[0] +=1
         dataset_fraction[0] +=1

for grph in test_dataset: 
    if grph.y == 1:
         test_fraction[1] +=1
         dataset_fraction[1] +=1 
    else:
         test_fraction[0] +=1
         dataset_fraction[0] +=1

print(f'Overall dataset percentage of label 1 = {dataset_fraction[1]/len(dataset)})')
print(f'Training dataset percentage of label 1 = {train_fraction} = {train_fraction[1]/len(train_dataset)}')
print(f'Validation dataset percentage of label 1 = {val_fraction} = {val_fraction[1]/len(val_dataset)}')
print(f'Test dataset percentage of label 1 = {test_fraction} = {test_fraction[1]/len(test_dataset)}')

### Input Graph: 

In [ ]:
x0, pos0, adj0 = torch.load(f'{model_name}_img0_data.pt')

In [ ]:
# Output of Embedding GNN
print(x0[0].shape)
x0[0]

In [ ]:
print(pos0[0].shape)
pos0[0]

In [ ]:
print(adj0[0].shape)
adj0[0]

In [ ]:
edge_index, _ = dense_to_sparse(adj0[0])
visualize_points(pos0[0].cpu(), edge_index)

### Graph After 1st Reduction

In [ ]:
x1_emb, x1_pool, pos1, adj1, s1= torch.load(f'{model_name}_img1_data.pt')

In [ ]:
# Output of Embedding GNN (adj0 @ x_0 @ w_gnn_emb)
print(x1_emb[0].shape)
x1_emb[0]

In [ ]:
# Output of Pooling GNN: adj_0 @ x_0 @ w_gnn_pool
print(s1[0].shape)
s1[0]

In [ ]:
# Output Coordinate Matrix (pos_out = softmax(s).t() @ pos_in)
print(pos1[0].shape)
pos1[0]

In [ ]:
# Output Feature Matrix (x_out = softmax(s).t() @ x_in)
print(x1_pool[0].shape)
x1_pool[0]

In [ ]:
# Output Adjacency Matrix = softmax(adj_out = softmax(s.t()) @ adj_in @ softmax(s))
print(adj1[0].shape)
adj1[0]

In [ ]:
edge_index, _ = dense_to_sparse(adj1[0])
visualize_points(pos1[0].cpu(), edge_index)

### Graph after 2nd reduction

In [ ]:
x2_emb, x2_pool, pos2, adj2, s2 = torch.load(f'{model_name}_img2_data.pt')

In [ ]:
# Output of Embedding GNN (adj1 @ x1_pool @ w_gnn_emb)
print(x2_emb[0].shape)
x2_emb[0]

In [ ]:
# Output of Pooling GNN: adj1 @ x1_pool @ w_gnn_pool), dim=1
print(s2[0].shape)
s2[0]

In [ ]:
# Output Coordinate Matrix (pos_out = softmax(s.t()) @ pos_in)
print(pos2[0].shape)
pos2[0]

In [ ]:
# Output Feature Matrix (x_out = softmax(s2).t() @ x2_emb)
print(x2_pool[0].shape)
x2_pool[0]

In [ ]:
# Output Adjacency Matrix (adj = softmax(s).T @ adj @ softmax(s)
print(adj2[0].shape)
adj2[0]

In [ ]:
edge_index, _ = dense_to_sparse(adj2[0])
visualize_points(pos2[0].cpu(), edge_index)

### Graph after 3rd reduction

In [ ]:
x3_emb, x3_pool, pos3, adj3, s3 = torch.load(f'{model_name}_img3_data.pt')

In [ ]:
# Output of Embedding GNN (adj_0 @ x_0 @ w_gnn_emb)
print(x3_emb[0].shape)
x3_emb[0]

In [ ]:
# Output of Pooling GNN: torch.softmax(adj_0 @ x_0 @ w_gnn_pool), dim=1)
print(s3[0].shape)
s3[0]

In [ ]:
# Output Coordinate Matrix (pos_out = softmax(s.t()) @ pos_in)
print(pos3[0].shape)
pos3[0]

In [ ]:
# Output Feature Matrix (x_out = softmax(s.t()) @ x_0)
print(x3_pool[0].shape)
x3_pool[0]

In [ ]:
# Output Adjacency Matrix (adj = softmax(s.t()) @ adj @ softmax(s)
print(adj3[0].shape)
adj3[0]

In [ ]:
edge_index, _ = dense_to_sparse(adj3[0])
visualize_points(pos3[0].cpu(), edge_index)